In [7]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

In [9]:
mp_holistic = mp.solutions.holistic

In [ ]:
def extract_keypoints_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    holistic = mp_holistic.Holistic(static_image_mode=False)
    keypoints_all_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)

        frame_keypoints = []

        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                frame_keypoints.extend([landmark.x, landmark.y, landmark.z])
        else:
            frame_keypoints.extend([0] * 33 * 3) 

        if results.left_hand_landmarks:
            for landmark in results.left_hand_landmarks.landmark:
                frame_keypoints.extend([landmark.x, landmark.y, landmark.z])
        else:
            frame_keypoints.extend([0] * 21 * 3)

        if results.right_hand_landmarks:
            for landmark in results.right_hand_landmarks.landmark:
                frame_keypoints.extend([landmark.x, landmark.y, landmark.z])
        else:
            frame_keypoints.extend([0] * 21 * 3)

        if results.face_landmarks:
            for landmark in results.face_landmarks.landmark:
                frame_keypoints.extend([landmark.x, landmark.y, landmark.z])
        else:
            frame_keypoints.extend([0] * 468 * 3)

        keypoints_all_frames.append(frame_keypoints)

    cap.release()
    holistic.close()
    return np.array(keypoints_all_frames)

In [13]:
def process_videos(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    video_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp4', '.avi', '.mov'))]

    for video_file in tqdm(video_files, desc="Processing videos"):
        video_path = os.path.join(input_dir, video_file)
        keypoints = extract_keypoints_from_video(video_path)
        output_path = os.path.join(output_dir, os.path.splitext(video_file)[0] + '.npy')
        np.save(output_path, keypoints)

In [15]:
input_directory = 'train'
output_directory = 'keypoints'
process_videos(input_directory, output_directory)

Processing videos:   0%|                                                                      | 0/7096 [00:00<?, ?it/s]D:\conda\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing videos: 100%|████████████████████████████████████████████████████████| 7096/7096 [16:11:44<00:00,  8.22s/it]
